<a href="https://colab.research.google.com/github/Josphat-Malombe/deep_learning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#getting data
import os
import requests
import zipfile
from pathlib import Path

data_path=Path("/data")
image_path=data_path/"pizza_steak_sushi"

if image_path.is_dir():
  print("directory exists")
else:
  print("creating the directory")
  image_path.mkdir(parents=True,exist_ok=True)

with open(data_path/"pizza_steak_sushi.zip", "wb") as f:
  request=requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("downloading the file")
  f.write(request.content)

with zipfile.ZipFile(data_path/"pizza_steak_sushi.zip", "r") as zip_ref:
  print("unzipping")
  zip_ref.extractall(image_path)

os.remove(data_path/ "pizza_steak_sushi.zip")

directory exists
downloading the file
unzipping


In [8]:

%%writefile main/data_setup.py
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
os.makedirs("main",exist_ok=True)
NUM_WORKERS=2


def create_dataloader(
  train_dir:str,
  test_dir:str,
  transform:transforms.Compose,
   batch_size:int,
   num_workers:int=NUM_WORKERS):


  train_data=datasets.ImageFolder(train_dir, transform=transform)
  test_data=datasets.ImageFolder(test_dir,transform=None)


  class_names=train_data.classes
  train_dataloader=DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True
  )
  test_dataloader=DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True
  )
  return train_dataloader,test_dataloader, class_names


Writing main/data_setup.py


In [9]:
%%writefile main/model.py

import torch
from torch import nn
class tinyVGG(nn.module):
  def __init__(self, input_shape:int,hidden_units:int,output_shape:int)->None:
    super().__init()
    self.conv_block1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool(kernel_size=2,stride=2)
        )
    self.conv_block1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool(kernel_size=2,stride=2)
        )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,out_features=output_shape)
    )
  def forward(self, x:torch.Tensor):
    x=self.conv_block1(x)
    x=self.conv_block2(x)
    x=self.classifier(x)
    return x






Writing main/model.py


In [10]:
%%writefile main/engine.py
import torch
from tqdm.auto import tqdm
from typing import List,Dict,Tuple
def train_step(model:torch.nn.Module,dataloader:torch.utils.data.DataLoader,criterion:torch.nn.Module,optimizer:torch.optim.optimizer):
  model.train()
  train_loss,train_acc=0,0
  for batch,(X,y) in enumerate(dataloader):
    y_pred=model(X)
    loss=criterion(y_pred,y)
    train_loss+=loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
    train_acc+=(y_pred_class==y).sum().item()/len(y_pred)
  train_loss=train_loss/len(dataloader)
  train_acc=train_acc/len(dataloader)
  return train_loss,train_acc
def test_step(model:torch.nn.Module,dataloader:torch.utils.data.DataLoader,criterion:torch.nn.Module):
  model.eval()
  test_loss,test_acc=0,0
  with torch.inference_mode():
    for batch,(X,y) in enumerate(dataloader):
      test_pred_logits=model(X)
      loss=criterion(test_pred_logits,y)
      test_loss+=loss.item()
      test_pred_labels=test_pred_logits.argmax(dim=1)
      test_acc+=((test_pred_labels==y).sum().item()/len(test_pred_labels))
  test_loss=test_loss/len(dataloader)
  test_acc=test_acc/len(dataloader)
  return test_loss,test_acc

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          criterion:torch.nn.Module,
          optimizer:torch.optim.Optimizers,
          epochs:int,)->Dict(str,List):

        results={"train_loss":[],
                 "train_acc":[],
                 "test_loss":[],
                 "test_acc":[]
                 }
        for epoch in tqdm(range(epochs)):
          train_loss,train_acc=train_step(model=model,dataloader=train_dataloader,criterion=criterion,optimizer=optimizer)
          test_loss,test_acc=test_step(model=model,dataloader=test_dataloader,criterion=criterion)

          print(
              f"epoch: {epoch+1} | "
              f"train_loss: {train_loss} | "
              f"train_acc: {train_acc} | "
              f"test_loss: {test_loss} | "
              f"test_acc: {test_acc} | "
          )

          results["train_loss"].append(train_loss)
          results["train_acc"].append(train_acc)
          results["test_loss"].append(test_loss)
          results["test_acc"].append(test_acc)
        return results



Writing main/engine.py


In [11]:
%%writefile main/utils.py
import torch
from pathlib import Path

def save_model(model:torch.nn.Module,
               target_dir:str,
               model_name:str):
  target_dir_path=Path(target_dir)
  target_dir_path.mkdir(parents=True,exist_ok=True)
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model name should have extension .pt , .pth"
  model_save_path=target_dir_path/model_name
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),f=model_save_path)

Writing main/utils.py


In [13]:
%%writefile main/train.py

import os
import torch
import dataset_up,engine,model,utils

from torchvision import transforms

NUM_EPOCHS=5
BATCH_SIZE=32
HIDDEN_UNIT=10
LEARNING_RATE=0.001

train_dir="data/pizza_steak_sushi/train"
test_dir="data/pizza_steak_sushi/test"

device="cuda" if torch.cuda.is_available() else "cpu"

data_transform=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

train_dataloader,test_dataloader,class_names=dataset_up.create_dataloader(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)
model=model.tinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNIT,
    output_shape=len(class_names)
)
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

engine.train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    epochs=NUM_EPOCHS,
)
utils.save_model(model=model,target_dir="models",model_name="granular_model.pth")



Overwriting main/train.py
